# Multi-tenant Chat with Papers - Query papers
## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

Weaviate Key:root-user-key
OpenAI API Key: sk-proj-iuwKF1Q94jnW
OpenAI URL: https://api.openai.com


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


## Vector search on tenants

In [3]:
papers = client.collections.get("Papers")

ten = papers.with_tenant("2212-10496")

response = ten.query.near_text(
    query="Unsupervised learning",
    limit=5,
)

for item in response.objects:
    print(item.properties["chunk"])

supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct learning algorithms. At the token level, generative large language models (LLM) pre- trained on large corpus have demonstrated strong natural language understanding (NLU) and gen- eration (NLG) capabilities (Brown et al., 2020; Chen et al., 2021; Rae et al., 2021; Hoffmann et al., 2022; Thoppilan et al., 2022; Chowdhery et al., 2022). At the document level, text (chunk) encoders pre-trained with contrastive objectives learn to encode document-document similarity into inner-product (Izacard et al., 2021; Gao and Callan, 2022). On top of these, one extra insight into LLM is borrowed: the LLMs further trained to follow instructions can zero-shot generalize to diverse un- seen instructions (Ouyang
many follow-up research generally consider the Transfer Learning setup where the dense re- triever is ﬁrst learned using a diverse and richly supervis

## Generative Search with tenants

In [4]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=3,
    single_prompt="What does the following text describe: {chunk}",
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generative.text, '\n')

supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct learning algorithms. At the token level, generative large language models (LLM) pre- trained on large corpus have demonstrated strong natural language understanding (NLU) and gen- eration (NLG) capabilities (Brown et al., 2020; Chen et al., 2021; Rae et al., 2021; Hoffmann et al., 2022; Thoppilan et al., 2022; Chowdhery et al., 2022). At the document level, text (chunk) encoders pre-trained with contrastive objectives learn to encode document-document similarity into inner-product (Izacard et al., 2021; Gao and Callan, 2022). On top of these, one extra insight into LLM is borrowed: the LLMs further trained to follow instructions can zero-shot generalize to diverse un- seen instructions (Ouyang
The text describes methods and advancements in self-supervised representation learning, particularly in the context of natural language processing (NL

In [5]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=3,
    grouped_task="Explain how unsupervised learning works. Use only the provided content.",
    grouped_properties=["chunk"]
)

for item in response.objects:
    print(item.properties["chunk"])

print(response.generative.text)

supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct learning algorithms. At the token level, generative large language models (LLM) pre- trained on large corpus have demonstrated strong natural language understanding (NLU) and gen- eration (NLG) capabilities (Brown et al., 2020; Chen et al., 2021; Rae et al., 2021; Hoffmann et al., 2022; Thoppilan et al., 2022; Chowdhery et al., 2022). At the document level, text (chunk) encoders pre-trained with contrastive objectives learn to encode document-document similarity into inner-product (Izacard et al., 2021; Gao and Callan, 2022). On top of these, one extra insight into LLM is borrowed: the LLMs further trained to follow instructions can zero-shot generalize to diverse un- seen instructions (Ouyang
many follow-up research generally consider the Transfer Learning setup where the dense re- triever is ﬁrst learned using a diverse and richly supervis

In [6]:
def paper_rag(paper_id, query, prompt):
    papers = client.collections.get("Papers")
    ten = papers.with_tenant(paper_id)

    response = ten.generate.near_text(
        query=query,
        limit=3,
        grouped_task=prompt + " Use only the provided content.",
        grouped_properties=["chunk"],
    )

    return {
        "title": response.objects[0].properties["title"],
        "source": [p.properties["chunk"] for p in response.objects],
        "generated": response.generative.text
    }

In [7]:
paper_rag(
    "2212-10496",
    "Unsupervised learning",
    "Explain how unsupervised learning works"
)

{'title': 'Precise Zero-Shot Dense Retrieval without Relevance Labels',
 'source': ['supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct learning algorithms. At the token level, generative large language models (LLM) pre- trained on large corpus have demonstrated strong natural language understanding (NLU) and gen- eration (NLG) capabilities (Brown et al., 2020; Chen et al., 2021; Rae et al., 2021; Hoffmann et al., 2022; Thoppilan et al., 2022; Chowdhery et al., 2022). At the document level, text (chunk) encoders pre-trained with contrastive objectives learn to encode document-document similarity into inner-product (Izacard et al., 2021; Gao and Callan, 2022). On top of these, one extra insight into LLM is borrowed: the LLMs further trained to follow instructions can zero-shot generalize to diverse un- seen instructions (Ouyang',
  'many follow-up research generally consider the Transfer Lear

In [8]:
papers = client.collections.get("Papers")
papers.tenants.get()

{'2212-10496': TenantOutput(name='2212-10496', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 '2401-00107': TenantOutput(name='2401-00107', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

## Close the client

In [9]:
client.close()